In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as L
from sklearn.model_selection import KFold
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import os
import gc
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files
_ = files.upload()

Saving SO2.zip to SO2.zip


In [ ]:
!unzip SO2.zip -d "SO2"
from IPython.display import clear_output
clear_output(wait=False)
!rm SO2.zip

In [ ]:
files = pd.read_csv("/content/SO2/SO2/files.csv")

In [ ]:
files

,Date,SatFile,GroundFile,AltGrid,Fold
0,2021-01-25,sat_2021-01-25.npy,g_2021-01-25.npy,alt_2021-01-25.npy,1
1,2021-02-21,sat_2021-02-21.npy,g_2021-02-21.npy,alt_2021-02-21.npy,2
2,2020-01-06,sat_2020-01-06.npy,g_2020-01-06.npy,alt_2020-01-06.npy,0
3,2021-01-07,sat_2021-01-07.npy,g_2021-01-07.npy,alt_2021-01-07.npy,3
4,2021-02-07,sat_2021-02-07.npy,g_2021-02-07.npy,alt_2021-02-07.npy,1
...,...,...,...,...,...
480,2020-04-08,sat_2020-04-08.npy,g_2020-04-08.npy,alt_2020-04-08.npy,0
481,2021-04-23,sat_2021-04-23.npy,g_2021-04-23.npy,alt_2021-04-23.npy,3
482,2020-11-20,sat_2020-11-20.npy,g_2020-11-20.npy,alt_2020-11-20.npy,2
483,2021-01-19,sat_2021-01-19.npy,g_2021-01-19.npy,alt_2021-01-19.npy,3


In [ ]:
def pearson(y_true,y_pred):
  if len(y_true.shape)!=1:
    true = []
    for i in range(y_true.shape[0]):
      true.extend(y_true[i])
    pred = []
    for i in range(y_pred.shape[0]):
      pred.extend(y_pred[i])
  else:
    true=y_true
    pred=y_pred
  return np.mean((np.array(true)-np.mean(true))*(np.array(pred)-np.mean(pred)))/(np.std(np.array(true))*np.std(np.array(pred)))

def pearsonCorrAvgDays(true, pred):
    assert true.shape == pred.shape, "true and pred must have the same shape, found {} and {}".format(true.shape, pred.shape)
    scores = []
    for i in range(true.shape[0]):
        scores.append(np.corrcoef(true[i], pred[i])[0, 1])
    return np.mean(scores), scores

def pearsonCorrAvgPixels(true,pred):
  scores = []
  for i in range(true.shape[1]):
    scores.append(pearson(true[:,i],pred[:,i]))
  return np.mean(scores),scores

In [ ]:
def loadData(df,satdir = "/content/SO2/SO2/satellite/",gdir = "/content/SO2/SO2/ground/"):
  X = []
  Y = []
  for i in range(df.shape[0]):
    factor = 46*(6.02214/6.023)*1e2
    sat = np.expand_dims(factor*np.load(os.path.join(satdir,df["SatFile"].iloc[i])),axis=2)
    ground = np.load(os.path.join(gdir,df["GroundFile"].iloc[i])).flatten()
    if not np.isnan(np.sum(sat)) and not np.isnan(np.sum(ground)):
      if not np.std(ground)==0:
        X.append(sat)
        Y.append(ground)
  return np.stack(X,axis=0),np.stack(Y,axis=0)

In [ ]:
!pip install torch
!pip install vit-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.5 MB/s eta 0:00:00


Vision Trnsformer

In [ ]:
#Build the model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms import Resize
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torchvision.models import vision_transformer
from vit_pytorch import ViT
from math import ceil

def build_model(X_train):
    #print(X_train.shape)
    # Calculate the image size and channels
    patch_size = 16  # Choose an appropriate patch size
    channels = X_train.shape[-1]
    #print("channels",channels)
    #desired_height = 48
    #desired_width = 48
    #transform = Resize((desired_height, desired_width))
    #X_train_transformed = transform(X_train)

    image_height = X_train.shape[1]  # Use the transformed image height
    image_width = X_train.shape[2]  # Use the transformed image width

    # Adjust the desired height and width to be divisible by the patch size
    resized_height = patch_size * (image_height // patch_size)
    resized_width = patch_size * (image_width // patch_size)

    assert resized_height > 0 and resized_width > 0, "Image dimensions are too small."

    model = ViT(
        image_size=(resized_height, resized_width),
        patch_size=patch_size,
        num_classes=3776,
        dim=64,
        depth=12,
        heads=8,
        mlp_dim=128,
        channels=channels
    )

    # Define the loss function
    criterion = nn.MSELoss()

    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    return model, criterion, optimizer

In [ ]:
import warnings
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import SGD
from vit_pytorch import ViT
from torchvision.transforms import RandomCrop, RandomHorizontalFlip, ColorJitter
from torchvision import transforms

# Model training
scores_list = []
rmses = []
maes = []
patch_size = 16
warnings.filterwarnings("ignore", category=UserWarning)

for fold in range(5):
    print("\nFold {}\n".format(fold))
    train_files = files[files["Fold"] != fold]
    val_files = files[files["Fold"] == fold]

    X_train, Y_train = loadData(train_files)
    X_val, Y_val = loadData(val_files)
    X_train_tensor = torch.from_numpy(X_train)  # Convert X_train to a PyTorch tensor
    print("Before resize of X_train ", X_train.shape)
    desired_height = 64
    desired_width = 48

    # Data augmentation
    transform = transforms.Compose([
        RandomCrop((desired_height, desired_width)),
        RandomHorizontalFlip(),
        ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        Resize((desired_height, desired_width))
    ])

    X_train = transform(torch.Tensor(X_train).permute(0, 3, 1, 2)).permute(0, 2, 3, 1)
    X_val = transform(torch.Tensor(X_val).permute(0, 3, 1, 2)).permute(0, 2, 3, 1)
    X_train_tensor = transform(X_train_tensor.permute(0, 3, 1, 2)).permute(0, 2, 3, 1)

    model, criterion, optimizer = build_model(X_train_tensor)
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)  # Example of using SGD optimizer

    if fold == 0:
        #print(model)
        pass

    train_dataset = torch.utils.data.TensorDataset(X_train.permute(0, 3, 1, 2), torch.Tensor(Y_train))
    val_dataset = torch.utils.data.TensorDataset(X_val.permute(0, 3, 1, 2), torch.Tensor(Y_val))

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8)

    patience = 5  # Number of epochs to wait for improvement
    best_val_loss = np.inf
    num_epochs_without_improvement = 0

    # Learning rate scheduler
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.1, verbose=True)

    for epoch in range(30):
        model.train()
        epoch_train_losses = []
        for inputs, targets in train_loader:
            optimizer.zero_grad()

            resized_inputs = inputs.clone()
            resized_inputs[:, :, :desired_height, :desired_width] = transform(inputs[:, :, :desired_height, :desired_width])

            resized_width = patch_size * (resized_inputs.shape[3] // patch_size)
            resized_inputs = resized_inputs[:, :, :, :resized_width]

            outputs = model(resized_inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            epoch_train_losses.append(loss.item())

        model.eval()
        with torch.no_grad():
            val_outputs = []
            for inputs, targets in val_loader:
                outputs = model(inputs)
                val_outputs.append(outputs)
            val_outputs = torch.cat(val_outputs, dim=0)

        rmse = mean_squared_error(Y_val, val_outputs, squared=False)



        # Update learning rate
        lr_scheduler.step(rmse)

        # Check for early stopping
        if rmse < best_val_loss:
            best_val_loss = rmse
            num_epochs_without_improvement = 0
        else:
            num_epochs_without_improvement += 1
            if num_epochs_without_improvement == patience:
                print("Early stopping criteria met. Stopping training.")
                break

    rmse1 = mean_squared_error(Y_val, val_outputs, squared=False)
    rmses.append(rmse1)
    mae = mean_absolute_error(Y_val, val_outputs)
    maes.append(mae)

    print("Fold {} RMSE Score: {}".format(fold, rmse))
    print("Fold {} MAE Score: {}".format(fold, mae))
    s, ls = pearsonCorrAvgDays(Y_val, val_outputs)
    print("Fold {} Pearson coeff avg over days: {}".format(fold, np.mean([i for i in ls if not pd.isnull(i)])))
    scores_list.append(ls)
    if fold != 4:
        del model
        _ = gc.collect()

print("\nCV RMSE Score: {}".format(np.mean(rmses)))
print("\nCV MAE Score: {}".format(np.mean(maes)))


Fold 0

Before resize of X_train  (345, 64, 59, 1)
Epoch 00006: reducing learning rate of group 0 to 1.0000e-03.
Epoch 00009: reducing learning rate of group 0 to 1.0000e-04.
Early stopping criteria met. Stopping training.
Fold 0 RMSE Score: 4.347727077638137
Fold 0 MAE Score: 3.021339604515683
Fold 0 Pearson coeff avg over days: 0.7745185759535933

Fold 1

Before resize of X_train  (335, 64, 59, 1)
Epoch 00011: reducing learning rate of group 0 to 1.0000e-03.
Early stopping criteria met. Stopping training.
Fold 1 RMSE Score: 4.913163140465493
Fold 1 MAE Score: 3.184273011052692
Fold 1 Pearson coeff avg over days: 0.7835889075592758

Fold 2

Before resize of X_train  (339, 64, 59, 1)
Epoch 00009: reducing learning rate of group 0 to 1.0000e-03.
Epoch 00018: reducing learning rate of group 0 to 1.0000e-04.
Early stopping criteria met. Stopping training.
Fold 2 RMSE Score: 4.71482699812772
Fold 2 MAE Score: 3.0767452615743576
Fold 2 Pearson coeff avg over days: 0.7799240040010824

Fold 

In [ ]:
inp = X_val[10][:,:,0]
#true = Y_val[10].reshape((X_train[0].shape[0],X_train[0].shape[1]))
true = Y_val[10].reshape((64,59))
pred = val_outputs[10].reshape((64,59))
#pred = val_outputs.view(X_val[10].shape[1], X_val[10].shape[2])

In [ ]:
import numpy as np
import folium
import branca.colormap as cm

ireland_bound = [-9.0893, -6.103800000000001,51.7995 , 55.004599999999996] #Irealand Extent
llcrn,urcrn = (-9.0893, 51.7995),(-6.103800000000001, 55.004599999999996) # Ireland bounds

# Create latitude and longitude grids
num_cols = int((urcrn[0] - llcrn[0]) // 0.05)
num_rows = int((urcrn[1] - llcrn[1]) // 0.05)
lat_coords = np.linspace(llcrn[1], urcrn[1], num_rows+1)
lon_coords = np.linspace(llcrn[0], urcrn[0], num_cols+1)
lat,lon = np.meshgrid(lon_coords, lat_coords)

true = np.array(true)
# Initialize a map
m = folium.Map(location=[53.0, -7.5], zoom_start=7, tiles='Stamen Terrain')

# Convert the numpy array to a GeoJSON-like structure
features = []
for i in range(num_rows):
    for j in range(num_cols):
        polygon = [
            [lat[i, j], lon[i, j]],
            [lat[i+1, j], lon[i+1, j]],
            [lat[i+1, j+1], lon[i+1, j+1]],
            [lat[i, j+1], lon[i, j+1]],
            [lat[i, j], lon[i, j]]
        ]
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [polygon]
            },
            'properties': {
                'value': true[i, j]  # Use the corresponding value from the 'true' array
            }
        }
        features.append(feature)

# Define a color scale for the 'true' values
vmin = np.min(true)
vmax = np.max(true)
colormap = cm.LinearColormap(colors=['blue', 'green', 'yellow', 'orange', 'red'], vmin=vmin, vmax=vmax)

# Add the GeoJSON layer to the map and set fill color based on 'true' values
folium.GeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['value']),
        'color': 'none',
        'fillOpacity': 0.6
    }
).add_to(m)


# Create a color bar legend with values
legend_html = '''
    <div style="
        position: fixed;
        bottom: 200px; left: 900px; width: 70px; height: 134px;
        background-color: white;
        z-index: 9999; font-size: 12px;
        border: 2px solid grey; padding: 2px;
    ">

'''

num_values = 5  # Number of values to display in the color bar legend
value_range = np.linspace(vmin, vmax, num_values)

for value in value_range:
    color = colormap(value)
    label = f'{int(value)}'  # Format the value as desired
    legend_html += f'<p><span style="background-color:{color};">&nbsp;&nbsp;&nbsp;&nbsp;</span> {label}</p>'

legend_html += '</div>'

# Add the color bar legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Crop the map to the bounds of the selected region
#m.fit_bounds([[llcrn[1], llcrn[0]], [urcrn[1], urcrn[0]]])
#m.fit_bounds(bounds)

# Save and display the map
m.save('map.html')

m

In [ ]:
import numpy as np
import folium
import branca.colormap as cm

ireland_bound = [-9.0893, -6.103800000000001,51.7995 , 55.004599999999996] #Irealand Extent
llcrn,urcrn = (-9.0893, 51.7995),(-6.103800000000001, 55.004599999999996) # Ireland bounds

# Create latitude and longitude grids
num_cols = int((urcrn[0] - llcrn[0]) // 0.05)
num_rows = int((urcrn[1] - llcrn[1]) // 0.05)
lat_coords = np.linspace(llcrn[1], urcrn[1], num_rows+1)
lon_coords = np.linspace(llcrn[0], urcrn[0], num_cols+1)
lat,lon = np.meshgrid(lon_coords, lat_coords)

pred = np.array(pred).astype(np.float64)
# Initialize a map
m = folium.Map(location=[53.0, -7.5], zoom_start=7, tiles='Stamen Terrain')

# Convert the numpy array to a GeoJSON-like structure
features = []
for i in range(num_rows):
    for j in range(num_cols):
        polygon = [
            [lat[i, j], lon[i, j]],
            [lat[i+1, j], lon[i+1, j]],
            [lat[i+1, j+1], lon[i+1, j+1]],
            [lat[i, j+1], lon[i, j+1]],
            [lat[i, j], lon[i, j]]
        ]
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [polygon]
            },
            'properties': {
                'value': pred[i, j]  # Use the corresponding value from the 'pred' aklnxrray
            }
        }
        features.append(feature)

# Define a color scale for the 'pred' values
vmin = np.min(pred)
vmax = np.max(pred)
colormap = cm.LinearColormap(colors=['blue', 'green', 'yellow', 'orange', 'red'], vmin=vmin, vmax=vmax)

# Add the GeoJSON layer to the map and set fill color based on 'pred' values
folium.GeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['value']),
        'color': 'none',
        'fillOpacity': 0.6
    }
).add_to(m)


# Create a color bar legend with values
legend_html = '''
    <div style="
        position: fixed;
        bottom: 200px; left: 900px; width: 70px; height: 134px;
        background-color: white;
        z-index: 9999; font-size: 12px;
        border: 2px solid grey; padding: 2px;
    ">

'''

num_values = 5  # Number of values to display in the color bar legend
value_range = np.linspace(vmin, vmax, num_values)

for value in value_range:
    color = colormap(value)
    label = f'{int(value)}'  # Format the value as desired
    legend_html += f'<p><span style="background-color:{color};">&nbsp;&nbsp;&nbsp;&nbsp;</span> {label}</p>'

legend_html += '</div>'

# Add the color bar legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Crop the map to the bounds of the selected region
#m.fit_bounds([[llcrn[1], llcrn[0]], [urcrn[1], urcrn[0]]])
#m.fit_bounds(bounds)

# Save and display the map
m.save('map.html')

m